In [1]:
# Import libraries
import pandas as pd
import numpy as np
import math

import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [25]:
geo_data = geopandas.read_file('https://raw.githubusercontent.com/electricitymap/electricitymap-contrib/master/web/geo/world.geojson')

In [3]:
arms_data = pd.read_csv('https://raw.githubusercontent.com/Kafkaese/ArmsTracker/main/Taro/data/ex_germany_2018.csv', index_col=0)

In [50]:
export_data = pd.read_csv('https://raw.githubusercontent.com/caatdata/eu-arms-export-data/master/export.csv', delimiter=';', header=None)

In [51]:
export_data = export_data.drop(columns = [11])

In [6]:
len(['year', 'embargo', 'source_country', 'source_country_name', 'destination_country', 'destination_country_name', 'region_code', 'region_name', 'rating', 'category', 'value'])

11

In [54]:
export_data.columns = ['year', 'embargo', 'source_country', 'source_country_name', 'destination_country', 'destination_country_name', 'region_code', 'region_name', 'rating', 'category', 'value']

In [55]:
export_data

,year,embargo,source_country,source_country_name,destination_country,destination_country_name,region_code,region_name,rating,category,value
0,1998,0,AT,Austria,,,,,,a,1605
1,1998,0,AT,Austria,,,,,,b,208741703
2,1998,0,AT,Austria,,,,,,d,13
3,1998,0,BE,Belgium,,,,,,a,1067
4,1998,0,BE,Belgium,,,,,,b,649671652
...,...,...,...,...,...,...,...,...,...,...,...
244821,2020,0,SE,Sweden,US,United States,AMN,North America,ML6,c,94701
244822,2020,0,SE,Sweden,US,United States,AMN,North America,ML8,a,15
244823,2020,0,SE,Sweden,US,United States,AMN,North America,ML8,b,13143315
244824,2020,0,SE,Sweden,US,United States,AMN,North America,ML8,c,14463613


In [56]:
export_total_data = export_data[export_data['year'] == 2020].groupby('source_country')[['value']].sum()

In [57]:
export_total_data = export_total_data.reset_index().rename(columns={'source_country': 'countryKey'})

In [58]:
export_total_data['countryKey'] = export_total_data['countryKey'].apply(lambda x: x.strip())

In [59]:
geo_data.head()

,zoneName,countryKey,countryName,geometry
0,XX,CY,Cyprus,"POLYGON ((33.90694 35.06887, 33.92494 35.27924..."
1,AE,AE,United Arab Emirates,"MULTIPOLYGON (((56.27756 25.62802, 56.15156 25..."
2,AF,AF,Afghanistan,"POLYGON ((74.54415 37.02230, 74.38214 37.12677..."
3,AL,AL,Albania,"POLYGON ((20.56881 41.87367, 20.50041 42.21141..."
4,AD,AD,Andorra,"POLYGON ((1.70822 42.50335, 1.72262 42.60925, ..."


In [60]:
data = geo_data.merge(export_total_data, how='left', on='countryKey')

In [61]:
data['total_mil'] = data['value'].apply(lambda x: round(x/1000000, 2))

In [62]:
data.head()

,zoneName,countryKey,countryName,geometry,value,total_mil
0,XX,CY,Cyprus,"POLYGON ((33.90694 35.06887, 33.92494 35.27924...",413337.0,0.41
1,AE,AE,United Arab Emirates,"MULTIPOLYGON (((56.27756 25.62802, 56.15156 25...",NaN,NaN
2,AF,AF,Afghanistan,"POLYGON ((74.54415 37.02230, 74.38214 37.12677...",NaN,NaN
3,AL,AL,Albania,"POLYGON ((20.56881 41.87367, 20.50041 42.21141...",NaN,NaN
4,AD,AD,Andorra,"POLYGON ((1.70822 42.50335, 1.72262 42.60925, ...",NaN,NaN


In [9]:
# data = pd.read_csv('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Live/master/data/neighborhood_data.csv')

In [10]:
# Read the geojson map file for Realtor Neighborhoods into a GeoDataframe object
#sf = geopandas.read_file('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Live/master/data/Realtor%20Neighborhoods.geojson')

# Set the Coordinate Referance System (crs) for projections
# ESPG code 4326 is also referred to as WGS84 lat-long projection
#sf.crs = {'init': 'epsg:4326'}

# Rename columns in geojson map file
#sf = sf.rename(columns={'geometry': 'geometry','nbrhood':'neighborhood_name', 'nid': 'subdist_no'}).set_geometry('geometry')

# Change neighborhood id (subdist_no) for correct code for Mount Davidson Manor and for parks
#sf.loc[sf['neighborhood_name'] == 'Mount Davidson Manor', 'subdist_no'] = '4n'
#sf.loc[sf['neighborhood_name'] == 'Golden Gate Park', 'subdist_no'] = '12a'
#sf.loc[sf['neighborhood_name'] == 'Presidio', 'subdist_no'] = '12b'
#sf.loc[sf['neighborhood_name'] == 'Lincoln Park', 'subdist_no'] = '12c'

#sf.sort_values(by=['subdist_no'])

In [11]:
# Create a function the returns json_data for the year selected by the user
def json_data(df):
    #yr = selectedYear
    
    # Pull selected year from neighborhood summary data
    #df_yr = neighborhood_data[neighborhood_data['year'] == yr]
    
    # Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
    #merged = pd.merge(sf, df_yr, on='subdist_no', how='left')
    
    # Fill the null values
    #values = {'year': yr, 'sale_price_count': 0, 'sale_price_mean': 0, 'sale_price_median': 0,
      #        'sf_mean': 0, 'price_sf_mean': 0, 'min_income': 0}
    #merged = merged.fillna(value=values)
    
    # Bokeh uses geojson formatting, representing geographical features, with json
    # Convert to json
    merged_json = json.loads(df.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data

In [12]:
# Define the callback function: update_plot
'''def update_plot(attr, old, new):
    # The input yr is the year selected from the slider
    yr = slider.value
    new_data = json_data(yr)
    
    # The input cr is the criteria selected from the select box
    cr = select.value
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, widgetbox(select), widgetbox(slider))
    curdoc().clear()
    curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = new_data
'''

"def update_plot(attr, old, new):\n    # The input yr is the year selected from the slider\n    yr = slider.value\n    new_data = json_data(yr)\n    \n    # The input cr is the criteria selected from the select box\n    cr = select.value\n    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]\n    \n    # Update the plot based on the changed inputs\n    p = make_plot(input_field)\n    \n    # Update the layout, clear the old document and display the new document\n    layout = column(p, widgetbox(select), widgetbox(slider))\n    curdoc().clear()\n    curdoc().add_root(layout)\n    \n    # Update the data\n    geosource.geojson = new_data\n"

In [13]:
# Create a plotting function
def make_plot(geosource):    
    # Set the format of the colorbar
    #min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
    #max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
    #field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

    # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    #color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

    # Create color bar.
    #format_tick = NumeralTickFormatter(format=field_format)
    #color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
    #border_line_color=None, location = (0, 0))

    # Create figure object.
    #verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

    p = figure(title = 'Title', 
             plot_height = 700, plot_width = 1200,
             toolbar_location = None)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False

    # Add patch renderer to figure. 
    p.patches('xs','ys', source = geosource,
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

    # Specify color bar layout.
    #p.add_layout(color_bar, 'right')

    # Add the hover tool to the graph
    p.add_tools(hover)
    return p

In [14]:
# This dictionary contains the formatting for the data in the plots
format_data = [('sale_price_count', 0, 100,'0,0', 'Number of Sales'),
               ('sale_price_mean', 500000, 4000000,'$0,0', 'Average Sales Price'),
               ('sale_price_median', 500000, 4000000, '$0,0', 'Median Sales Price'),
               ('sf_mean', 500, 5000,'0,0', 'Average Square Footage'),
               ('price_sf_mean', 0, 2000,'$0,0', 'Average Price Per Square Foot'),
               ('min_income', 50000, 600000,'$0,0', 'Minimum Income Required')
              ]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

In [15]:
format_df

,field,min_range,max_range,format,verbage
0,sale_price_count,0,100,"0,0",Number of Sales
1,sale_price_mean,500000,4000000,"$0,0",Average Sales Price
2,sale_price_median,500000,4000000,"$0,0",Median Sales Price
3,sf_mean,500,5000,"0,0",Average Square Footage
4,price_sf_mean,0,2000,"$0,0",Average Price Per Square Foot
5,min_income,50000,600000,"$0,0",Minimum Income Required


In [16]:
#json_data(2018)

In [47]:
# Input geojson source that contains features for plotting for:
# initial year 2018 and initial criteria sale_price_median
geosource = GeoJSONDataSource(geojson = json_data(data))
#input_field = 'sale_price_median'

# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Add hover tool
hover = HoverTool(tooltips = [ ('Name','@countryName'), ('Total Exports 2020 (Million EUR)', '@total_mil')])

# Call the plotting function
p = make_plot(geosource)

# Make a slider object: slider 
#slider = Slider(title = 'Year',start = 2009, end = 2018, step = 1, value = 2018)
#slider.on_change('value', update_plot)

# Make a selection object: select
#select = Select(title='Select Criteria:', value='Median Sales Price', options=['Median Sales Price', 'Minimum Income Required',
#                                                                               'Average Sales Price', 'Average Price Per Square Foot',
 #                                                                              'Average Square Footage', 'Number of Sales'])
#select.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
# Display the current document
layout = column(p)
curdoc().add_root(layout)